In [2]:
import scanpy as sc

In [3]:
import rapids_singlecell as rapids

In [4]:
import sklearn

In [5]:
sklearn.__version__

'1.4.2'

In [6]:
# ad_path_organoids = '/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks.h5ad'
ad_paths = ['/mnt/f/workspace/theislab/mubind/data/noack_2022/merged_scATAC_integrated_cicero_faye_chong.h5ad',
            '/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks.h5ad',
             '/mnt/f/workspace/theislab/mubind/data/pancreatic_endocrinogenesis/pancreas_multiome_2022_processed_atac.h5ad',
             '/mnt/f/workspace/theislab/mubind/data/pbmc/processed_laura_2023.h5ad',
             '/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks.h5ad']

In [7]:
ad_by_path = {}

In [8]:
import pandas as pd
import os

# the ratio of peaks over cells e.g. 3 : 1
ratio_feature_over_cells = 3

overwrite = False
for ad_path in ad_paths:
    # if not 'laura' in ad_path:
    #     continue

    for selection in ['random', 'episcanpy']:
        if selection == 'random':
            print('')
        print('')
        print(selection, ad_path)
        outdir = os.path.join(os.path.dirname(ad_path), selection)
        print(outdir)
        if not os.path.exists(outdir):
            os.makedirs(outdir)

        # last = False
        for n_sample_cells in [500, 1000, 2000, 5000, 10000, 'ALL']:

            # if n_sample_cells == 'ALL':
            #     continue

            n_sample_peaks = int(n_sample_cells * ratio_feature_over_cells) if n_sample_cells is not 'ALL' else 'ALL'
            adata = None

            outname = os.path.basename(ad_path.replace('.h5ad', '_obs%s_var%s.h5ad' % (n_sample_cells, n_sample_peaks)))
            outpath = os.path.join(outdir, outname)

            print(os.path.exists(outpath), outpath)

            if os.path.exists(outpath) and not overwrite:
                continue

            print('attempting to read')

            # continue
            # assert False
            ad = sc.read_h5ad(ad_path) if not ad_path in ad_by_path else ad_by_path[ad_path]

            # if n_sample_cells is not None:
            # if n_sample_cells < ad.shape[0]:
            print(n_sample_cells)
            adata = sc.pp.subsample(ad,
                                    n_obs=ad.shape[0] if isinstance(n_sample_cells, str) else n_sample_cells,
                                    copy=True,
                                    random_state=ad.shape[0] if isinstance(n_sample_cells, str) else n_sample_cells)
            # else:
            #     adata = ad
            #     last = True
            # else:
            #     adata = ad

            print('after cells selection', adata.shape)

            # sc.pp.highly_variable_genes(adata, n_top_genes=2500)
            # using episcanpy
            print('selection rule', selection)
            if selection == 'episcanpy':
                acc = (adata.X != 0).sum(axis=0) / adata.shape[0]
                acc_score = abs(.5 - acc)
                adata.var['acc_score'] = acc_score.A.T
                adata.var['acc_score_rank'] = adata.var['acc_score'].rank(ascending=False)
                # plt.hist(atac.var['acc_score'], bins=35)
                var_sample = adata.var.sort_values('acc_score_rank').index[:n_sample_peaks if n_sample_peaks is not 'ALL' else adata.shape[1]]
                adata = adata[:,adata.var_names.isin(var_sample)].copy()
            elif selection == 'random':
                # obs_sample = pd.Series(adata.obs_names).sample(n_sample_cells)
                var_sample = pd.Series(adata.var_names).sample(n_sample_peaks if isinstance(n_sample_peaks, int) else adata.shape[1])
                # var_sample = atac.var.sort_values('acc_score_rank').index[:n_sample_peaks]
                # adata = adata[adata.obs_names.isin(obs_sample),adata.var_names.isin(var_sample)].copy()
                adata = adata[:,adata.var_names.isin(var_sample)].copy()

            print('after peaks selection', adata.shape)

            print('sum of reads', adata.X.sum())
            if adata.X.sum() == 0:
                print('skip...not too many peaks after selection...')
                continue

            # adata.shape
            sc.pp.neighbors(adata) #  method='rapids')
            sc.tl.umap(adata, init_pos="random", random_state=1) #  method='rapids')

            # out of memory in very large cases
            # rapids.pp.neighbors(adata) #  method='rapids')
            # rapids.tl.umap(adata, init_pos="random", random_state=1) #  method='rapids')

            # adata.X.A.max()
            # sc.pl.umap(adata, color='celltype')
            assert outpath != ad_path

            # adata.shape
            adata.write(outpath, compression='lzf')

            # if last:
            #     break

<>:24: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:65: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:24: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:65: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_31068/4144545223.py:24: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  n_sample_peaks = int(n_sample_cells * ratio_feature_over_cells) if n_sample_cells is not 'ALL' else 'ALL'
/tmp/ipykernel_31068/4144545223.py:65: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  var_sample = adata.var.sort_values('acc_score_rank').index[:n_sample_peaks if n_sample_peaks is not 'ALL' else adata.shape[1]]




random /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks.h5ad
/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/random
True /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/random/RNA_ATAC_metacells_sce_peaks_obs500_var1500.h5ad
True /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/random/RNA_ATAC_metacells_sce_peaks_obs1000_var3000.h5ad
True /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/random/RNA_ATAC_metacells_sce_peaks_obs2000_var6000.h5ad
True /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/random/RNA_ATAC_metacells_sce_peaks_obs5000_var15000.h5ad
True /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/random/RNA_ATAC_metacells_sce_peaks_obs10000_var30000.h5ad
False /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/random/RNA_ATAC_metacells_sce_peaks_obsALL_varALL.h5ad
attempting to read
ALL
